In [1]:
import transformers
import os
import pandas as pd
import natsort
import numpy as np
from PIL import Image
import time

## Import/Load Dependencies:
# timm==0.4.12
# transformers==4.15.0  #using it as replacment of 4.22.0
# fairscale==0.4.4
# pycocoevalcap

if os.getcwd() != "/home/nitesh/Documents/MY_THESIS/MAMI/BLIP":
    %cd BLIP
print(os.getcwd())

print('transformer version:', transformers.__version__)
# !git clone https://github.com/salesforce/BLIP
# Ref: https://github.com/salesforce/BLIP

/home/nitesh/Documents/MY_THESIS/MAMI/BLIP
/home/nitesh/Documents/MY_THESIS/MAMI/BLIP
transformer version: 4.15.0


In [2]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    
#  watch -n 2 nvidia-smi

cuda
NVIDIA GeForce RTX 3070 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
start_time = time.time()

# Load the training text data
train2 = pd.read_csv('../training1.csv', delimiter='\t')


# Load the training image data
IDs = []  #set1
images = []
directory = '../TRAINING'   # directory where we have images 
filenames = natsort.natsorted(os.listdir(directory))  

# Correct the images as some images are not having 3 channels.Omit such images.
# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
#         ID = int(filename[:-4])
        ID = filename
        pathname = os.path.join(directory, filename)
        im = Image.open(pathname)
#         im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
           print("This is 1 channel, so we omit it", imnp.shape, filename)
           continue
        IDs.append(ID)
        images.append(imnp)
        

# Align the image and textual data and extract where we have both available 
def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))

# Example usage
list1 = IDs
list2 = list(train2.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(list1, list2)
print('len of common strings', len(common_strings))

sorted_ids = natsort.natsorted(common_strings)




def load_image(image_size, directory, img_id, device):

    # Correct the images as some images are not having 3 channels.Omit such images.
    # get the ids from the images, where images are having three channels; omit images if channels != 3
#     for i, filename in enumerate(img_id):
    if img_id.endswith(".jpg"):
        ID_0 = img_id
        pathname = os.path.join(directory, img_id)
        print(pathname)
        im = Image.open(pathname)
        w,h=im.size
        im.resize((w//5,h//5))
        transform = transforms.Compose([
            transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
            ]) 
        image = transform(im).unsqueeze(0).to(device)  
#             display(image)
            

    return ID_0, image 


def image_generator(sorted_ids, batch_size, directory):
    image_size = 384
    num_images = len(sorted_ids)
    num_batches = (num_images + batch_size - 1) // batch_size

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = min((batch_idx + 1) * batch_size, num_images)
        batch_ids = sorted_ids[start_idx:end_idx]

        batch_images = []
        batch_img_ids = []
        for filename in batch_ids:
            ID_0, imss = load_image(image_size, directory, filename, device=device)
            batch_images.append(imss)
            batch_img_ids.append(ID_0)

        yield batch_img_ids, torch.stack(batch_images)

# Usage example
batch_size = 5
imggen=image_generator(sorted_ids, batch_size, directory)
image_size = 384


from models.blip import blip_decoder
blip_model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth'
model = blip_decoder(pretrained=blip_model_url, image_size=image_size, vit='base')
model.eval()
model = model.to(device)



caption_df = pd.DataFrame(columns=['file_name', 'text'])

with torch.no_grad():
    for batch_images in imggen:
        for img_id, imss in zip(batch_images[0], batch_images[1]):
            # Generate caption
            caption = model.generate(imss, sample=False, num_beams=3, max_length=20, min_length=5)
#             caption_text = caption[0]['sequence']
            print(img_id, caption[0])
            
            # Append to the dataframe
            caption_df = caption_df.append({'file_name': img_id, 'text': caption[0]}, ignore_index=True)
            

# Compute the runtime
runtime = time.time() - start_time

# Print the runtime
print("Total runtime: {:.2f} seconds".format(runtime))

This is 1 channel, so we omit it (479, 640) 149.jpg
This is 1 channel, so we omit it (640, 521) 202.jpg
This is 1 channel, so we omit it (470, 640) 1893.jpg
This is 1 channel, so we omit it (570, 640) 1997.jpg
This is 1 channel, so we omit it (640, 471) 2436.jpg
This is 1 channel, so we omit it (427, 640) 3713.jpg
This is 1 channel, so we omit it (626, 640) 3906.jpg
This is 1 channel, so we omit it (640, 640) 3969.jpg
This is 1 channel, so we omit it (640, 566) 5051.jpg
This is 1 channel, so we omit it (640, 584) 5639.jpg
This is 1 channel, so we omit it (640, 467) 6313.jpg
This is 1 channel, so we omit it (640, 375) 9166.jpg
This is 1 channel, so we omit it (640, 543) 11618.jpg
This is 1 channel, so we omit it (594, 640) 11715.jpg
len of common strings 9986
reshape position embedding from 196 to 576
load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth
../TRAINING/1.jpg
../TRAINING/2.jpg
../TRAINING/3.jpg
../TRAINING/

2023-07-12 15:43:55.669343: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


1.jpg milk and cheese on a table
2.jpg a halo halo halo halo halo halo halo halo halo halo halo halo halo halo halo
3.jpg a man kneeling next to a deer
4.jpg the floor plan for a bedroom
5.jpg a woman with glasses on
../TRAINING/6.jpg
../TRAINING/7.jpg
../TRAINING/8.jpg
../TRAINING/9.jpg
../TRAINING/10.jpg
6.jpg a woman with a beard and a man with a beard
7.jpg a man talking on the phone
8.jpg a man in a suit sitting at a table
9.jpg a man in a suit and tie
10.jpg a panda bear sitting on top of a tree stump
../TRAINING/11.jpg
../TRAINING/12.jpg
../TRAINING/13.jpg
../TRAINING/14.jpg
../TRAINING/15.jpg
11.jpg a glass of beer with a capt
12.jpg a man playing cards on a table
13.jpg a woman in a pink bikini
14.jpg trump and his wife
15.jpg two people in a kitchen
../TRAINING/16.jpg
../TRAINING/17.jpg
../TRAINING/18.jpg
../TRAINING/19.jpg
../TRAINING/21.jpg
16.jpg a woman sitting in a car
17.jpg a person in front of a screen
18.jpg a man holding a gun in front of a building
19.jpg a man in 

165.jpg a seal seal with the capt that says,'birthday card pretend not to note
../TRAINING/166.jpg
../TRAINING/167.jpg
../TRAINING/169.jpg
../TRAINING/170.jpg
../TRAINING/171.jpg
166.jpg a woman with a green face
167.jpg a woman in a black dress with the capt that reads we'd'er
169.jpg a man in a tuxed tuxed tuxed tuxed tuxed tuxed
170.jpg a woman sitting on a couch
171.jpg two people standing at a counter
../TRAINING/173.jpg
../TRAINING/174.jpg
../TRAINING/175.jpg
../TRAINING/176.jpg
../TRAINING/177.jpg
173.jpg a group of children playing in the grass
174.jpg two women and a cat
175.jpg a man smoking a cigarette
176.jpg a person in a hospital bed
177.jpg the floor plan for a two bedroom apartment
../TRAINING/178.jpg
../TRAINING/179.jpg
../TRAINING/180.jpg
../TRAINING/181.jpg
../TRAINING/182.jpg
178.jpg a cat on a tree in the snow
179.jpg a woman with a sign that says it's huge to hide up
180.jpg two men holding up a sign
181.jpg a fish in the ocean
182.jpg a computer screen with a bun

312.jpg a person standing on a stage in front of a crowd
313.jpg a girl on a boat with the capt that says nobody women
315.jpg a man holding an ice cream cone
316.jpg a woman holding a glass of beer
../TRAINING/317.jpg
../TRAINING/318.jpg
../TRAINING/319.jpg
../TRAINING/320.jpg
../TRAINING/321.jpg
317.jpg a man with a sword in his hand
318.jpg a man with a horse
319.jpg a man with a sword in his hand and the capt that says,'cy
320.jpg a girl with a volleyball ball on her head stock photo
321.jpg a man holding a woman's face
../TRAINING/322.jpg
../TRAINING/323.jpg
../TRAINING/326.jpg
../TRAINING/327.jpg
../TRAINING/328.jpg
322.jpg two people walking down the street
323.jpg two people sitting on a subway train
326.jpg a black and white photo with the words like girl on period
327.jpg some of the characters in the game
328.jpg a man sitting at a table
../TRAINING/329.jpg
../TRAINING/330.jpg
../TRAINING/331.jpg
../TRAINING/332.jpg
../TRAINING/333.jpg
329.jpg a family sitting on a couch wit

...

11306.jpg a man sitting in a car with the capt that reads remember the day before what
../TRAINING/11307.jpg
../TRAINING/11308.jpg
../TRAINING/11309.jpg
../TRAINING/11310.jpg
../TRAINING/11311.jpg
11307.jpg a man and a little boy
11308.jpg two people shaking each other people
11309.jpg a girl with a sad look on her face
11310.jpg a hole in the ground
11311.jpg a man in a suit and tie, with the capt that reads,'you
../TRAINING/11312.jpg
../TRAINING/11314.jpg
../TRAINING/11316.jpg
../TRAINING/11317.jpg
../TRAINING/11319.jpg
11312.jpg a young boy with the capt that says, when you see your face, you
11314.jpg a woman holding a shovel
11316.jpg a group of people standing in front of a building
11317.jpg a man in a kitchen with the capt that says,'i'm o
11319.jpg two kangaroos playing with each other animals
../TRAINING/11321.jpg
../TRAINING/11322.jpg
../TRAINING/11323.jpg
../TRAINING/11325.jpg
../TRAINING/11326.jpg
11321.jpg an older man sitting at a desk with a laptop
11322.jpg a woman in 

11438.jpg a man with a hat and glasses
11439.jpg an office space with a man in a tie and suspends
11440.jpg a woman with tattoos on her stomach
11441.jpg a woman with her mouth open
11443.jpg two men laying in bed
../TRAINING/11444.jpg
../TRAINING/11445.jpg
../TRAINING/11446.jpg
../TRAINING/11447.jpg
../TRAINING/11450.jpg
11444.jpg a naked girl leaning on a pole
11445.jpg a group of people at a party
11446.jpg two women holding signs in front of a pool
11447.jpg a man in a suit and tie
11450.jpg two men in a room
../TRAINING/11451.jpg
../TRAINING/11453.jpg
../TRAINING/11454.jpg
../TRAINING/11455.jpg
../TRAINING/11457.jpg
11451.jpg a man with a smile on his face
11453.jpg a cartoon drawing of a man in a car with the capt that reads when you
11454.jpg a thomas the tank train
11455.jpg a pokemon pokemon meme
11457.jpg two people in yellow suits
../TRAINING/11458.jpg
../TRAINING/11459.jpg
../TRAINING/11460.jpg
../TRAINING/11462.jpg
../TRAINING/11464.jpg
11458.jpg a bunch of red and white b

11576.jpg a young girl with the capt that says,'what do you think??
11577.jpg a man with a red hair and glasses
11578.jpg a bunch of people with different facial expressions
11579.jpg a dragon with the capt that reads,'my armor is your name '
11580.jpg a young boy with his mouth open
../TRAINING/11581.jpg
../TRAINING/11583.jpg
../TRAINING/11584.jpg
../TRAINING/11585.jpg
../TRAINING/11586.jpg
11581.jpg two people sitting in the grass
11583.jpg a woman with green leaves
11584.jpg three women in a room with text that reads, i'm'm '
11585.jpg a man and a little girl
11586.jpg a woman in a santa hat taking a selfie
../TRAINING/11587.jpg
../TRAINING/11589.jpg
../TRAINING/11590.jpg
../TRAINING/11591.jpg
../TRAINING/11592.jpg
11587.jpg an orange flower with a capt that reads speaking my girlfriend
11589.jpg a man in a blue shirt
11590.jpg a woman sitting on a couch
11591.jpg a man in a military uniform
11592.jpg a woman in underwear and panties
../TRAINING/11593.jpg
../TRAINING/11594.jpg
../TR

11718.jpg a woman in a black outfit
11719.jpg a lizard eating something on the roof
11720.jpg a soldier with a gun in the water
11721.jpg a woman sitting on the ground
11722.jpg the floor plan for a studio apartment
../TRAINING/11723.jpg
../TRAINING/11724.jpg
../TRAINING/13000.jpg
../TRAINING/13003.jpg
../TRAINING/13006.jpg
11723.jpg a man drinking a beer
11724.jpg a man wearing a hat and glasses
13000.jpg a fat woman in a black dress
13003.jpg a woman laying on a towel next to a pool
13006.jpg a woman smiling with the capt
../TRAINING/13009.jpg
../TRAINING/13018.jpg
../TRAINING/13021.jpg
../TRAINING/13027.jpg
../TRAINING/13033.jpg
13009.jpg a woman in a bikini
13018.jpg a woman in a bikini on the beach
13021.jpg two kids eating at a table
13027.jpg a woman laying on the beach
13033.jpg a woman in a red shirt
../TRAINING/13036.jpg
../TRAINING/13039.jpg
../TRAINING/13042.jpg
../TRAINING/13045.jpg
../TRAINING/13051.jpg
13036.jpg a woman in a white shirt
13039.jpg an old man with a bottle

13432.jpg a man with blood on his face
13435.jpg a woman in a blue bikini
13438.jpg a woman lifting a barbell
13441.jpg a man with a mustache and mustache
13444.jpg a woman in a purple shirt
../TRAINING/13447.jpg
../TRAINING/13453.jpg
../TRAINING/13456.jpg
../TRAINING/13462.jpg
../TRAINING/13465.jpg
13447.jpg three women drinking cocktails
13453.jpg a woman with long hair
13456.jpg a woman in black and white
13462.jpg two people playing curling
13465.jpg a person with a quote on it
../TRAINING/13468.jpg
../TRAINING/13471.jpg
../TRAINING/13477.jpg
../TRAINING/13480.jpg
../TRAINING/13483.jpg
13468.jpg a man and a woman dancing
13471.jpg a woman with a broom in her mouth
13477.jpg a girl holding a rope
13480.jpg a man in a blue shirt
13483.jpg a man and woman in a car
../TRAINING/13486.jpg
../TRAINING/13489.jpg
../TRAINING/13492.jpg
../TRAINING/13498.jpg
../TRAINING/13501.jpg
13486.jpg a person holding a can of soda
13489.jpg a person holding a remote control
13492.jpg a man with a bottle

In [24]:
os.chdir("/home/nitesh/Documents/MY_THESIS/MAMI/BLIP")
caption_df.to_csv('train_BLIP_captions.csv', index=False)

In [5]:
# TEST DATA IMAGES LOADING


### LOAD TEST DATA

test1 = pd.read_csv('../Test.csv', delimiter='\t')

test_labels = pd.read_csv('../test_labels.txt', delimiter='\t', header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

merged_test = pd.merge(test1, test_labels, on='file_name', how='inner')


# Sort the dataframe with natural ordering of the IDs
merged_test['prefix_file_name'] = merged_test['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
merged_test1 = merged_test.sort_values(by='prefix_file_name', ascending=True)
merged_test1


# # train = train.rename(columns={'Text Transcription': 'text'})
test2 = merged_test1.rename(columns={'Text Transcription': 'text'})

TEST_IDs = []  #set1
TEST_images = []
TEST_directory = '../TEST'   # directory where we have images 
test_filenames = natsort.natsorted(os.listdir(TEST_directory))  

# Correct the images as some images are not having 3 channels.Omit such images.
# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(test_filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
#         ID = int(filename[:-4])
        TEST_ID = filename
        pathname = os.path.join(TEST_directory, filename)
        im = Image.open(pathname)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        TEST_imnp = np.array(im)
        if len(TEST_imnp.shape) != 3:
#             print("This is 1 channel, so we omit it", TEST_imnp.shape, filename)
            continue
        TEST_IDs.append(TEST_ID)
        TEST_images.append(TEST_imnp)
        
        
# Align the image and textual data and extract where we have both available 
def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))

# Example usage
TEST_list1 = TEST_IDs
TEST_list2 = list(test2.file_name)  #from the text file where we have text description 
TEST_common_strings = get_common_strings(TEST_list1, TEST_list2)
print('len of common strings', len(TEST_common_strings))

TEST_sorted_ids = natsort.natsorted(TEST_common_strings)
# len(TEST_sorted_ids)

len of common strings 1000


In [8]:
# Load the test image data
TEST_directory = '../TEST' 
test_sorted_ids = TEST_sorted_ids  # Sorted IDs of the test images

test_caption_df = pd.DataFrame(columns=['file_name', 'text'])

with torch.no_grad():
    for img_id in test_sorted_ids:
        image_size = 384
        # Load the test image
        IDs_test,test_image = load_image(image_size, TEST_directory, img_id, device=device)

        # Generate caption
        caption = model.generate(test_image, sample=False, num_beams=3, max_length=20, min_length=5)
        caption_text = caption[0]

        # Append to the dataframe
        test_caption_df = test_caption_df.append({'file_name': img_id, 'text': caption_text}, ignore_index=True)


../TEST/15001.jpg
../TEST/15002.jpg
../TEST/15004.jpg
../TEST/15005.jpg
../TEST/15006.jpg
../TEST/15008.jpg
../TEST/15009.jpg
../TEST/15010.jpg
../TEST/15011.jpg
../TEST/15012.jpg
../TEST/15014.jpg
../TEST/15016.jpg
../TEST/15017.jpg
../TEST/15018.jpg
../TEST/15019.jpg
../TEST/15020.jpg
../TEST/15021.jpg
../TEST/15022.jpg
../TEST/15024.jpg
../TEST/15025.jpg
../TEST/15026.jpg
../TEST/15027.jpg
../TEST/15031.jpg
../TEST/15033.jpg
../TEST/15034.jpg
../TEST/15035.jpg
../TEST/15036.jpg
../TEST/15037.jpg
../TEST/15038.jpg
../TEST/15041.jpg
../TEST/15043.jpg
../TEST/15044.jpg
../TEST/15045.jpg
../TEST/15047.jpg
../TEST/15048.jpg
../TEST/15049.jpg
../TEST/15050.jpg
../TEST/15051.jpg
../TEST/15055.jpg
../TEST/15056.jpg
../TEST/15057.jpg
../TEST/15058.jpg
../TEST/15062.jpg
../TEST/15065.jpg
../TEST/15067.jpg
../TEST/15068.jpg
../TEST/15070.jpg
../TEST/15072.jpg
../TEST/15073.jpg
../TEST/15074.jpg
../TEST/15078.jpg
../TEST/15081.jpg
../TEST/15083.jpg
../TEST/15085.jpg
../TEST/15088.jpg
../TEST/15

../TEST/15692.jpg
../TEST/15693.jpg
../TEST/15694.jpg
../TEST/15695.jpg
../TEST/15696.jpg
../TEST/15697.jpg
../TEST/15698.jpg
../TEST/15699.jpg
../TEST/15700.jpg
../TEST/15701.jpg
../TEST/15703.jpg
../TEST/15704.jpg
../TEST/15705.jpg
../TEST/15707.jpg
../TEST/15711.jpg
../TEST/15712.jpg
../TEST/15713.jpg
../TEST/15714.jpg
../TEST/15716.jpg
../TEST/15717.jpg
../TEST/15718.jpg
../TEST/15719.jpg
../TEST/15720.jpg
../TEST/15721.jpg
../TEST/15722.jpg
../TEST/15723.jpg
../TEST/15725.jpg
../TEST/15726.jpg
../TEST/15727.jpg
../TEST/15728.jpg
../TEST/15729.jpg
../TEST/15731.jpg
../TEST/15732.jpg
../TEST/15734.jpg
../TEST/15735.jpg
../TEST/15737.jpg
../TEST/15738.jpg
../TEST/15742.jpg
../TEST/15743.jpg
../TEST/15745.jpg
../TEST/15746.jpg
../TEST/15747.jpg
../TEST/15748.jpg
../TEST/15749.jpg
../TEST/15750.jpg
../TEST/15752.jpg
../TEST/15753.jpg
../TEST/15754.jpg
../TEST/15755.jpg
../TEST/15756.jpg
../TEST/15758.jpg
../TEST/15759.jpg
../TEST/15760.jpg
../TEST/15763.jpg
../TEST/15765.jpg
../TEST/15

../TEST/16314.jpg
../TEST/16315.jpg
../TEST/16317.jpg
../TEST/16320.jpg
../TEST/16323.jpg
../TEST/16324.jpg
../TEST/16325.jpg
../TEST/16327.jpg
../TEST/16328.jpg
../TEST/16329.jpg
../TEST/16332.jpg
../TEST/16334.jpg
../TEST/16335.jpg
../TEST/16336.jpg
../TEST/16337.jpg
../TEST/16338.jpg
../TEST/16339.jpg
../TEST/17000.jpg
../TEST/17002.jpg
../TEST/17003.jpg
../TEST/17004.jpg
../TEST/17005.jpg
../TEST/17006.jpg
../TEST/17007.jpg
../TEST/17008.jpg
../TEST/17011.jpg
../TEST/17012.jpg
../TEST/17013.jpg
../TEST/17015.jpg
../TEST/17016.jpg
../TEST/17017.jpg
../TEST/17018.jpg
../TEST/17020.jpg
../TEST/17021.jpg
../TEST/17022.jpg
../TEST/17023.jpg
../TEST/17024.jpg
../TEST/17025.jpg
../TEST/17026.jpg
../TEST/17027.jpg
../TEST/17028.jpg
../TEST/17029.jpg
../TEST/17030.jpg
../TEST/17031.jpg
../TEST/17032.jpg
../TEST/17033.jpg
../TEST/17034.jpg
../TEST/17036.jpg
../TEST/17037.jpg
../TEST/17038.jpg
../TEST/17039.jpg
../TEST/17041.jpg
../TEST/17042.jpg
../TEST/17043.jpg
../TEST/17044.jpg
../TEST/17

In [25]:
# os.chdir("/home/nitesh/Documents/MY_THESIS/MAMI/BLIP")
test_caption_df.to_csv('test_BLIP_captions.csv', index=False)

References:

https://github.com/jasonnoy/BLIP/blob/main/models/blip.py

https://github.com/pharmapsychotic


https://github.com/jasonnoy?tab=repositories

https://medium.com/analytics-vidhya/dcgan-implementing-deep-convolutional-generative-adversarial-network-in-tensorflow-idiot-41b2ddf9eddb

https://github.com/opetrova/SemiSupervisedPytorchGAN/blob/master/README.md

https://github.com/pharmapsychotic/clip-interrogator